In [1]:
import os
import sys
import django

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "True"
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "LabTimetablingAPI.settings")
django.setup()

In [2]:
default_config = {
    "semester": 1,
    "local_search": {
        "algorithm": "simulated_annealing",
        "config": {
            "neighborhood": {
                "algorithm": "random_swap",
                "random_swap": {
                    "neighborhood_size": 100
                },
                "random_range_swap": {
                    "neighborhood_size_factor": 0.1,
                    "range_size_factor": 0.1
                },
                "distance_swap": {
                    "distance_percentage": 0.1
                },
                "swap": False
            },
            "fitness": {
                "group_assignment_conflict": {
                    "max_threshold": 3,
                    "conflict_penalty": 1
                },
                "assistant_distribution": {
                    "max_group_threshold": 15,
                    "max_shift_threshold": 50,
                    "group_penalty": 1,
                    "shift_penalty": 1
                }
            },
            "simulated_annealing": {
                "initial_temperature": 50,
                "cooling_rate": 0.1,
                "max_iteration": 25,
                "max_time": 5,
                "max_iteration_without_improvement": 10,
            },
            "tabu_search": {
                "tabu_list_size": 50,
                "max_iteration": 1000,
                "max_time": 60,
                "max_iteration_without_improvement": 100,
                "max_time_without_improvement": 5
            }
        }
    },
    "algorithm": {
        "algorithm": "genetic_local_search",
        "config": {
            "max_iteration": 500,
            "population_size": 25,
            "elitism_size": 2,
            "fitness": {
                "group_assignment_conflict": {
                    "max_threshold": 3,
                    "conflict_penalty": 1
                },
                "assistant_distribution": {
                    "max_group_threshold": 15,
                    "max_shift_threshold": 25,
                    "group_penalty": 1,
                    "shift_penalty": 1
                }
            },
            "operator": {
                "selection": {
                    "roulette_wheel": True,
                    "tournament": False,
                    "elitism": True,
                    "tournament_size": 5
                },
                "crossover": {
                    "single_point": True,
                    "two_point": False,
                    "uniform": False,
                    "crossover_probability": 0.1,
                    "uniform_probability": 0.5
                },
                "mutation": {
                    "swap": True,
                    "shift": False,
                    "random": False,
                    "mutation_probability": 0.1
                },
                "repair": {
                    "time_slot": True
                }
            }
        }
    }
}




In [3]:
from scheduling_algorithm.data_parser import ModuleData
from scheduling_algorithm.algorithms import (
    GeneticAlgorithm,
    GeneticLocalSearch
)
from scheduling_algorithm.utils.solution_generator import SolutionGenerator
from scheduling_algorithm.factory import WeeklyFactory

In [4]:
modules = ModuleData.get_modules_by_semester(1)

In [5]:
generator = SolutionGenerator.from_data(default_config)
weekly_generator = SolutionGenerator.from_data(default_config)

Loading configuration...
Configuration loaded successfully.
Creating Genetic Local Search Algorithm from configuration
Population Size:  25
Max Iteration:  500
Creating FitnessManager with fitness functions:  [Fitness(name=GroupAssignmentConflictFitness), Fitness(name=AssistantDistributionFitness)]
Elitism:  Selection(name=ElitismSelection)
Configured selection functions:  [Selection(name=RouletteWheelSelection), Selection(name=ElitismSelection)]
Configuring crossover operator:  [Crossover(name=SinglePointCrossover)]
Configuring mutation operator:  [Mutation(name=SwapMutation)]
Configuring repair operator:  [Repair(name=RepairTimeSlot)]
Creating FitnessManager with fitness functions:  [Fitness(name=GroupAssignmentConflictFitness), Fitness(name=AssistantDistributionFitness)]
Creating NeighborhoodManager with neighborhood: Neighborhood(name=RandomSwapNeighborhood)
Creating Simulated Annealing
Local Search Algorithm:  Search(name=SimulatedAnnealing)
Loading configuration...
Configuration 

In [6]:
import cProfile as profile
import pstats
from pstats import SortKey

profiler = profile.Profile()
profiler.enable()
solution = weekly_generator.generate_solution_weekly_test()
profiler.disable()

Generating population for module 1 week 1
Generating population for module 1 week 2
Generating population for module 1 week 3
Generating population for module 1 week 4
Generating population for module 1 week 5
Module 1 week 5 has no population, all the remaining chapter are already assigned on previous weeks
Starting the schedule generation algorithm...


In [8]:
solution

Chromosome(length=288, fitness=35)

In [9]:
stats = pstats.Stats(profiler).sort_stats(SortKey.TIME)
stats.print_stats()

         102204416 function calls (101845077 primitive calls) in 68.926 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
 13064397   12.523    0.000   12.523    0.000 {method 'copy' of 'dict' objects}
   179700   10.481    0.000   21.120    0.000 d:\Projects\Kuliah\skripsi\jte-lab-timetabling\LabTimetablingAPI\scheduling_algorithm\operator\repair\time_slot_repair.py:45(__call__)
   180600   10.140    0.000   12.373    0.000 d:\Projects\Kuliah\skripsi\jte-lab-timetabling\LabTimetablingAPI\scheduling_algorithm\fitness_function\group_assignment_conflict.py:26(__call__)
   180600   10.008    0.000   14.877    0.000 d:\Projects\Kuliah\skripsi\jte-lab-timetabling\LabTimetablingAPI\scheduling_algorithm\fitness_function\assistant_distribution.py:27(__call__)
 14500277    3.967    0.000    3.967    0.000 d:\Projects\Kuliah\skripsi\jte-lab-timetabling\LabTimetablingAPI\scheduling_algorithm\operator\repair\time_slot_repair.py:66(_is_ti

In [ ]:
solution1 = generator.generate_solution_weekly_test()

Generating population for module 1


In [20]:
solution1

Chromosome(length=288, fitness=116)